In [1]:
import sklearn as sk
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, precision_recall_fscore_support
from pylab import plot, plt
import keras
from keras.layers import Dense, Input
from keras.models import Model

/Users/ekremguzelyel/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pd.read_csv("Final Data.csv")
data.drop(columns="SEQN", inplace=True)
data["TELOMEAN"]= data["TELOMEAN"].astype(int)

data["Weight H.BMI"] = data["Weight H.BMI"].fillna(0.0).astype(int)
# data["SMQ.SMQ040"] = data["SMQ.SMQ040"].astype(str)

# data["SMQ.SMD075"] = data["SMQ.SMD075"].astype(int)
# data["PA.PAQ180"] = data["PA.PAQ180"].astype(str)
# data["PA.PAQ520"] = data["PA.PAQ520"].astype(str)
# data["DU.DUQ100"] = data["DU.DUQ100"].astype(str)
# data["Gender and Age.RIAGENDR"] = data["Gender and Age.RIAGENDR"].astype(str)

categoricals = data[["SMQ.SMQ040","PA.PAQ180","PA.PAQ520","Gender and Age.RIAGENDR"]].astype(str)
integers = data[["Weight H.BMI","SMQ.SMD075","ALQ.ALQ130","Gender and Age.RIDAGEYR"]].astype(int)
categoricals, integers

(     SMQ.SMQ040 PA.PAQ180 PA.PAQ520 Gender and Age.RIAGENDR
 0             3         1         3                       2
 1             3         3         3                       1
 2             3         1         3                       2
 3             3         1         3                       2
 4             3         2         2                       2
 5             3         3         3                       1
 6             3         2         3                       2
 7             3         2         3                       1
 8             3         2         3                       2
 9             3         2         3                       2
 10            3         1         1                       1
 11            3         3         1                       1
 12            3         3         3                       2
 13            1         2         3                       1
 14            1         3         3                       1
 15            1        

In [3]:
le = LabelEncoder()
cat_one_hot = categoricals.apply(le.fit_transform).astype(str)
cat_one_hot.shape, cat_one_hot.values

((1749, 4), array([['2', '0', '2', '1'],
        ['2', '2', '2', '0'],
        ['2', '0', '2', '1'],
        ...,
        ['0', '1', '1', '1'],
        ['0', '0', '2', '0'],
        ['0', '1', '0', '0']], dtype=object))

In [4]:
data_combined = pd.concat([cat_one_hot,integers],axis=1)
data_combined

,SMQ.SMQ040,PA.PAQ180,PA.PAQ520,Gender and Age.RIAGENDR,Weight H.BMI,SMQ.SMD075,ALQ.ALQ130,Gender and Age.RIDAGEYR
0,2,0,2,1,20,0,3,20
1,2,2,2,0,23,0,4,20
2,2,0,2,1,30,0,6,20
3,2,0,2,1,20,0,5,20
4,2,1,1,1,30,0,4,20
5,2,2,2,0,24,0,10,20
6,2,1,2,1,23,0,5,20
7,2,1,2,0,26,0,7,20
8,2,1,2,1,21,0,6,20
9,2,1,2,1,27,0,1,20


In [5]:
X = data_combined.values
y = data.TELOMEAN.values

x_train, x_test, y_train, y_test = train_test_split(X, y,test_size=1/3, random_state=42)
len(x_train), len(x_test)

(1166, 583)

In [21]:
X[0], x_test[0]

(array(['2', '0', '2', '1', 20, 0, 3, 20], dtype=object),
 array(['2', '2', '2', '0', 37, 0, 2, 35], dtype=object))

### Applying Neural Nets

In [156]:
# Different combinations tested.
input_layer = Input(shape=(x_train.shape[1],))

hidden1 = Dense(32,activation='relu',)(input_layer)

# cnn1 = Conv1D(128, 5, activation='tanh')(hidden1)
# pool1 = MaxPooling1D(5)(cnn1)
# cnn2 = Conv1D(128, 5, activation='sigmoid')(pool1)
# pool2 = GlobalMaxPooling1D()(cnn2)
hidden2 = Dense(16, activation='relu')(hidden1)
hidden3 = Dense(8, activation='relu')(hidden2)
output_layer = Dense(1, activation='linear')(hidden3)
model = Model(inputs=input_layer, outputs=output_layer)


model.compile(loss='mean_squared_error', metrics=['mean_squared_error'], optimizer='adam',)
print("Model Compiled")

Model Compiled


In [157]:
model.fit(x_train, y_train,epochs=200)

Epoch 1/200
1166/1166 [==============================] - 0s 359us/step - loss: 35154669.6123 - mean_squared_error: 35154669.6123
Epoch 2/200
1166/1166 [==============================] - 0s 52us/step - loss: 35040830.2093 - mean_squared_error: 35040830.2093
Epoch 3/200
1166/1166 [==============================] - 0s 46us/step - loss: 34768003.8388 - mean_squared_error: 34768003.8388
Epoch 4/200
1166/1166 [==============================] - 0s 49us/step - loss: 33976110.4974 - mean_squared_error: 33976110.4974
Epoch 5/200
1166/1166 [==============================] - 0s 49us/step - loss: 31980033.7050 - mean_squared_error: 31980033.7050
Epoch 6/200
1166/1166 [==============================] - 0s 54us/step - loss: 27823598.0343 - mean_squared_error: 27823598.0343
Epoch 7/200
1166/1166 [==============================] - 0s 43us/step - loss: 20675962.3431 - mean_squared_error: 20675962.3431
Epoch 8/200
1166/1166 [==============================] - 0s 44us/step - loss: 11411635.0515 - mean_squa

1166/1166 [==============================] - 0s 73us/step - loss: 1459027.7406 - mean_squared_error: 1459027.7406
Epoch 65/200
1166/1166 [==============================] - 0s 57us/step - loss: 1456382.2592 - mean_squared_error: 1456382.2592
Epoch 66/200
1166/1166 [==============================] - 0s 61us/step - loss: 1450796.2940 - mean_squared_error: 1450796.2940
Epoch 67/200
1166/1166 [==============================] - 0s 62us/step - loss: 1443953.0740 - mean_squared_error: 1443953.0740
Epoch 68/200
1166/1166 [==============================] - 0s 40us/step - loss: 1438118.0878 - mean_squared_error: 1438118.0878
Epoch 69/200
1166/1166 [==============================] - 0s 46us/step - loss: 1433601.1653 - mean_squared_error: 1433601.1653
Epoch 70/200
1166/1166 [==============================] - 0s 46us/step - loss: 1424367.4370 - mean_squared_error: 1424367.4370
Epoch 71/200
1166/1166 [==============================] - 0s 44us/step - loss: 1421569.8852 - mean_squared_error: 1421569.88

1166/1166 [==============================] - 0s 51us/step - loss: 1178264.4325 - mean_squared_error: 1178264.4325
Epoch 128/200
1166/1166 [==============================] - 0s 53us/step - loss: 1176455.4583 - mean_squared_error: 1176455.4583
Epoch 129/200
1166/1166 [==============================] - 0s 43us/step - loss: 1174147.6490 - mean_squared_error: 1174147.6490
Epoch 130/200
1166/1166 [==============================] - 0s 44us/step - loss: 1168991.0806 - mean_squared_error: 1168991.0806
Epoch 131/200
1166/1166 [==============================] - 0s 42us/step - loss: 1164941.7654 - mean_squared_error: 1164941.7654
Epoch 132/200
1166/1166 [==============================] - 0s 38us/step - loss: 1178891.7704 - mean_squared_error: 1178891.7704
Epoch 133/200
1166/1166 [==============================] - 0s 39us/step - loss: 1159649.1207 - mean_squared_error: 1159649.1207
Epoch 134/200
1166/1166 [==============================] - 0s 39us/step - loss: 1157783.0806 - mean_squared_error: 115

Epoch 190/200
1166/1166 [==============================] - 0s 54us/step - loss: 1013082.8163 - mean_squared_error: 1013082.8163
Epoch 191/200
1166/1166 [==============================] - 0s 81us/step - loss: 1009796.5819 - mean_squared_error: 1009796.5819
Epoch 192/200
1166/1166 [==============================] - 0s 77us/step - loss: 1006604.1439 - mean_squared_error: 1006604.1439
Epoch 193/200
1166/1166 [==============================] - 0s 80us/step - loss: 1003535.5352 - mean_squared_error: 1003535.5352
Epoch 194/200
1166/1166 [==============================] - 0s 76us/step - loss: 1009960.0824 - mean_squared_error: 1009960.0824
Epoch 195/200
1166/1166 [==============================] - 0s 44us/step - loss: 1003492.0760 - mean_squared_error: 1003492.0760
Epoch 196/200
1166/1166 [==============================] - 0s 43us/step - loss: 1000144.1008 - mean_squared_error: 1000144.1008
Epoch 197/200
1166/1166 [==============================] - 0s 42us/step - loss: 993909.9848 - mean_squar

In [17]:
model.predict(x_test)

array([[ 7037.909 ],
       [ 5262.8394],
       [ 4566.901 ],
       [ 3924.6248],
       [ 3902.3218],
       [ 4529.7334],
       [ 5779.2437],
       [ 3928.121 ],
       [ 5477.992 ],
       [ 6376.346 ],
       [ 4278.365 ],
       [ 6728.792 ],
       [ 5378.618 ],
       [ 5846.135 ],
       [ 6020.9854],
       [ 6514.6445],
       [ 5536.2964],
       [ 5495.85  ],
       [ 5422.62  ],
       [ 4562.5254],
       [ 4695.698 ],
       [ 5854.2505],
       [ 7996.1997],
       [ 5737.0513],
       [ 5274.332 ],
       [ 5674.423 ],
       [ 4544.694 ],
       [ 7171.164 ],
       [ 6079.6553],
       [ 6221.1855],
       [ 5004.26  ],
       [ 5868.0674],
       [ 6807.184 ],
       [ 5476.08  ],
       [ 5598.2627],
       [ 5624.8105],
       [ 6936.0684],
       [ 5931.229 ],
       [ 5302.033 ],
       [ 5722.6436],
       [ 6866.4517],
       [ 6326.2734],
       [ 5901.469 ],
       [ 4034.074 ],
       [ 5333.5176],
       [ 5302.833 ],
       [ 6100.883 ],
       [ 4871

In [158]:
model.evaluate(x_test,y_test)

583/583 [==============================] - 0s 159us/step


[1036620.1788968696, 1036620.1788968696]

In [152]:
weight_1 = model.get_weights()[0]*-1
# model.get_weights()[5]
weight_1
weights=model.get_weights()
for i in range(len(weights[0])):
    print(i)
weights

0
1
2
3
4
5
6
7


[array([[-0.1549131 ,  0.40583548,  0.709981  ,  0.25760856,  0.68595153,
         -0.30174986,  0.33745044, -0.28097153,  0.43523866,  0.02107808,
          0.87007713,  0.13773823,  0.6236122 , -0.29153076,  0.6670897 ,
          0.50488967,  0.58333826,  0.60273373, -0.3795066 ,  0.8155268 ,
          0.6969647 ,  0.40178034,  0.23881581,  0.11243969,  0.79920894,
          0.18820697, -1.85478   ,  0.29373908,  0.37192506, -2.1601057 ,
         -0.2918548 ,  0.38752687],
        [ 0.35900372,  1.2935873 ,  0.79668766,  0.9034636 ,  0.74766695,
         -0.3043407 ,  0.9066454 ,  0.08390099,  0.94071144,  0.06379026,
          2.0665915 , -0.12208441,  1.1649407 ,  0.32566965,  1.4129869 ,
          1.1732785 ,  1.463387  ,  1.2566371 ,  0.17246324,  1.9491075 ,
          1.5318663 ,  1.1766741 ,  0.72015923, -0.04639035,  0.78156674,
          0.31482887, -3.059754  , -0.08536586, -0.37147632, -2.7240734 ,
          0.3131612 ,  1.2327884 ],
        [-0.02525014,  0.25457937,  0.69

In [36]:
model.predict(np.array([['2', '2', '2', '0', 37, 0, 2, 35],]))[0]

array([7037.9097], dtype=float32)

In [175]:
11500-model.predict(np.array([['0','2','2','0',23,0,0,75],]))[0]

array([4457.3257], dtype=float32)

In [153]:
model1 = model.save
model1_weights = model.save_weights

In [183]:
model.save(filepath="model2.h5")
model.save_weights(filepath="model2_weights.h5")

#### Trials

In [181]:
11500-model.predict(np.array([['0','2','2','0',23,0,0,60],]))[0]

array([5799.2217], dtype=float32)